# SPARK ACTIVITY 1

### ACTIVITY QUESTION

For the file movie_lines.txt, your task is to see what is the most frequent “first”
Word in the movie lines. For this activity you do not have to remove punctuation
But you will get 5 bonus points if you do so! Do all this using your rdd...

In [1]:
import s3fs
fs = s3fs.S3FileSystem()

In [2]:
strLoc = 's3://686springbucket-emmanuel/movie-dialog-corpus/'

In [3]:
fs.ls(strLoc)

['686springbucket-emmanuel/movie-dialog-corpus/.DS_Store',
 '686springbucket-emmanuel/movie-dialog-corpus/README.txt',
 '686springbucket-emmanuel/movie-dialog-corpus/chameleons.pdf',
 '686springbucket-emmanuel/movie-dialog-corpus/movie_characters_metadata.txt',
 '686springbucket-emmanuel/movie-dialog-corpus/movie_conversations.txt',
 '686springbucket-emmanuel/movie-dialog-corpus/movie_lines.txt',
 '686springbucket-emmanuel/movie-dialog-corpus/movie_titles_metadata.txt',
 '686springbucket-emmanuel/movie-dialog-corpus/raw_script_urls.txt']

In [4]:
# Opening up the movie_lines.txt file
with fs.open(strLoc + 'movie_lines.txt', 'rb') as f:
    data = f.read()

In [5]:
#the code below is used to convert the file read to latin-1, we often also converts to utf-8
data = data.decode('latin-1')

Creating a function to remove punctuation as directed by the question

In [6]:
lsLines = data.split('\n')

# Defining useful functions that would be applied to RDD

def remove_punc(string):
    punc = '''!()-[]{};:'"\,<>./?@#%^&*_~'''
    for a in string:
        if a in punc:
            string = string.replace(a, "")
    return string

def Func(lines):
    lines = lines.lower()
    lines = lines.split(' +++$+++ ')[4]
    return lines

In [7]:
# To see if the file was read, decoded and if the split by new line was done properly
lsLines[0:4]

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?']

#### Setting up pyspark

In [8]:
import pyspark

In [9]:
from pyspark import SparkContext

In [10]:
sc = SparkContext()

In [11]:
#the last line is junk so get rid of it ([0:-1])
# then parallelize the list

lineRDD = sc.parallelize(lsLines[0:-1])

#### Applying our defined function to RDDs to solve the task given

In [12]:
lineRDD1 = lineRDD.map(Func)

In [13]:
lineRDD2 = lineRDD1.map(remove_punc)

#### Getting the First spoken word by an actor for each line

In [14]:
#use map to split each line by the delimiter then get the first word spoken by each actor out of there,
#finally create an item with the first spoken word by an actor and a 1
wordRDD = lineRDD2.map(lambda line:(line.split(' ')[0],1))

In [15]:
print(wordRDD.collect()[0:4])

[('they', 1), ('they', 1), ('i', 1), ('she', 1)]


In [16]:
#reduce by key
wordRDD = wordRDD.reduceByKey(lambda x,y:x+y)

In [17]:
print(wordRDD.collect()[0:4])

[('i', 28164), ('no', 9527), ('im', 7060), ('like', 605)]


In [18]:
#sort 
wordRDD = wordRDD.sortBy(lambda word: -word[1])

In [19]:
print(wordRDD.collect()[0:4])

[('i', 28164), ('you', 15365), ('what', 12046), ('no', 9527)]


In [20]:
sc.stop()